# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [12]:
import json

In [13]:
with open("pa1_data/courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [14]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [15]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])
        course["enrolled"]=int(course["enrolled"])

In [16]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to answer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

# Student Code Below Here:

A) How many faculty taught COSI courses last year?

In [17]:
instructors=set(course["instructor"] for course in courses)
print(len(instructors))


904


B) What is the total number of students taking COSI courses last year?

In [18]:
students= sum([course["enrolled"] for course in courses if course["subject"].lower()=="cosi"])
print(students)

2223


C) What was the median size of a COSI course last year (counting only those courses with at least 10 students)?

In [19]:
eligibleCourses=sorted([course ["enrolled"] for course in courses if course["enrolled"]>=10])
median = eligibleCourses[int(len(eligibleCourses)/2)]
print(median)

20


D) Create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.

In [20]:
subjects=set(course["subject"] for course in courses)
#top_courses=[tuple(E,S) for E in courses for S in courses]
top_courses=[]
for subject in subjects:
    temp=0
    for course in courses:
        if course["subject"]==subject:
            temp+=course["enrolled"]
    top_courses.append(tuple((temp,subject)))

#top_courses=[tuple((course["enrolled"],subject)) for subject in subjects for course in courses if course["subject"]==subject]

#top_courses=sorted(top_courses)
#print(top_courses)
print([top_courses[i] for i in range(len(top_courses)-1,len(top_courses)-11,-1)])


[(92, 'BCBP'), (109, 'RBIF'), (383, 'CHIN'), (300, 'IGS'), (13, 'IIM'), (7, 'EAS'), (1704, 'PSYC'), (61, 'RIDT'), (18, 'ECS/ENG'), (43, 'LALS')]


E) Do the same as in (d) but print the top 10 subjects in terms of number of courses offered.

F) Do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject.

G) List the top 20 faculty in terms of number of students they taught.

H) List the top 20 courses in terms of number of students taking that course (where you combine different sections and semesters, i.e. just use the subject and course number).

I) Create your own interesting question (each team member creates their own) and use Python to answer that question.

Ben: Which professor teaches the most COSI courses?

Jason:

Kayla: Which many classes in the econ department had under 50 students?

Kevin: How many faculty taught COSI courses with atleast 10 students last year?

In [21]:
instructors={course["instructor"] for course in courses if course["enrolled"]>=10}
print(len(instructors))

652


Mason: 